Install requeriments

In [1]:
!pip install pydub

!apt install sox

!pip install tflearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 7 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,715 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com/ubunt

Download lapsbm dataset for test

In [2]:
import os 
base = 'man/dev'
# download dataset https://www.openslr.org/61/
!wget -c -q --show-progress -O ./man.tar.gz https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-3/zh-TW.tar.gz
!mkdir man
os.chdir('man') 
!tar -xzf ../man.tar.gz

os.chdir('..')
!ls

./man.tar.gz        100%[===================>]   1.12G  22.5MB/s    in 52s     
man  man.tar.gz  sample_data


In [3]:
!ls man/

clips  dev.tsv	invalidated.tsv  other.tsv  test.tsv  train.tsv  validated.tsv


In [4]:
!head -n 60 man/train.tsv 

client_id	path	sentence	up_votes	down_votes	age	gender	accent
729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a	common_voice_zh-TW_17377831.mp3	我們特別回鄉下	2	0	thirties	male	
729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a	common_voice_zh-TW_17377841.mp3	是歷史上的第二次	2	0	thirties	male	
729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a	common_voice_zh-TW_17377844.mp3	簡單來說	2	1	thirties	male	
729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a	common_voice_zh-TW_17377846.mp3	在田裡也需幫忙	2	0	thirties	male	
729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a	common_voice_zh-TW_17377848.mp3	婚後一年生了個女嬰	2	1	thirties	male

In [0]:
from pydub import AudioSegment as audio
import scipy.io.wavfile

def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms
 
    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0  # ms
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold:
        trim_ms += chunk_size
 
    return trim_ms

def remove_silence(file_path,out_aux='aux_silence.wav'):
    sound = audio.from_mp3(file_path)
    start_trim = detect_leading_silence(sound)
    end_trim = detect_leading_silence(sound.reverse())
    duration = len(sound)
    trimmed_sound = sound[start_trim:duration-end_trim]
    trimmed_sound.export(out_aux, format="wav")

In [6]:

import pandas as pd 
import librosa 
import numpy as np

df = pd.read_csv("man/train.tsv",sep='\t')
id_ant = ''

audio_dir = 'man/clips'
wav = []
waves = []
count = 0 
loc= 0
for ind in df.index: 
     if count >= 20:
       break
     id,gender,file = df['client_id'][ind], df['gender'][ind], df['path'][ind]
     if id == id_ant or id_ant == '':
         remove_silence(os.path.join(audio_dir,file))
         wav = wav+list((librosa.load('aux_silence.wav',sr=44100)[0]))
         print('cadastrando:',id)  
     else:
          librosa.output.write_wav('aux_save.wav',np.array(wav),sr=44100)
          waves.append([audio.from_wav('aux_save.wav'),loc])
          loc +=1
          remove_silence(os.path.join(audio_dir,file))
          wav = list((librosa.load('aux_silence.wav',sr=44100)[0]))
          print('cadastrado:',id_ant)  
          count+=1 
     id_ant = id
loc +=1
librosa.output.write_wav('aux_save.wav',np.array(wav),sr=44100)
waves.append([audio.from_wav('aux_save.wav'),loc])    

cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadastrando: 729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a6130251d096b3fcde600a0b146fa0a02d14a8b4acd96f06db6e7a2d63042c85ffc43d0ba58b1fc392a
cadast

In [7]:
waves = waves

amostras_cadastradas = [] #armazenara as amostras de cada locutor ( uma cada locutor)
amostras_novas= [] # armazena as amostras de teste, serão varias por locutor a ideia é que com as amostras cadastradas seja possivel identificar o locutor dessas novas amostras
flag = 0 
#segmenta todos os áudios em 5 segundos.
print(len(waves))
for wave in waves: 
  

  begin = 0
  end = 5
  flag = 0
  
  
  while (end*1) < int(wave[0].duration_seconds):
    try:
        
      segment = wave[0][begin*1000:end*1000]
      # o áudio é exportado pois é necessario ler e extrair o mfcc com a biblioteca librosa( mesma utilizada no treinamento do modelo)  
      segment.export('aux' + '.wav', 'wav')
      y, sr = librosa.load('aux.wav')   
      if(flag == 0):
        amostras_cadastradas.append([librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13),int(wave[1])])
        flag =1
      else:
        amostras_novas.append([librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13),int(wave[1])])
      
      
      os.system('rm aux.wav')
      begin = begin + 1
      end = end + 1
      
    except:
      print('erro')
      begin = begin + 1
      end = end + 1


print('Numero de locutores:', len(amostras_cadastradas), ' Numero de novas amostras a serem testadas:', len(amostras_novas))



21
Numero de locutores: 20  Numero de novas amostras a serem testadas: 415


In [8]:
import tflearn

#Download Weights
!wget -O ./saver.zip https://www.dropbox.com/s/b19xt2wu3th9p36/Save-Models-Speaker-Diarization.zip?dl=0
!unzip saver.zip

#Create model for restore
encoder = tflearn.input_data(shape=[None, 13,int(216)])
encoder = tflearn.dropout(encoder,0.9) #10 % drop - 90% -> 80
encoder = tflearn.dropout(encoder,0.2)# 80 % drop
encoder = tflearn.fully_connected(encoder, 40,activation='crelu')
decoder = tflearn.fully_connected(encoder, int(572), activation='linear')
net = tflearn.regression(decoder, optimizer='adam', learning_rate=0.0007,loss='mean_square', metric=None)#categorical_crossentropy
model = tflearn.DNN(net, tensorboard_verbose=0,tensorboard_dir='tflearn_logs')


model.load('./Save-Models/Model3-Best-40loc.tflearn')

encoding_model = tflearn.DNN(encoder, session=model.session)# used for extract embedding in encoder layer








--2019-12-30 19:11:47--  https://www.dropbox.com/s/b19xt2wu3th9p36/Save-Models-Speaker-Diarization.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b19xt2wu3th9p36/Save-Models-Speaker-Diarization.zip [following]
--2019-12-30 19:11:47--  https://www.dropbox.com/s/raw/b19xt2wu3th9p36/Save-Models-Speaker-Diarization.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc4a8697d872ef9eacf13f3b3d9.dl.dropboxusercontent.com/cd/0/inline/AvMnn8mXliVnniO6xr7k2eh8buC2ZRX3aDp37CU1Ti0KPF8Dw5CroIvQ_AFAGYjHwgKqSPmrK2t5vm5PZ5Q5pknTNJHflK2N5VkUlbiQJJOD_g/file# [following]
--2019-12-30 19:11:48--  https://ucc4a8697d872ef9eacf13f3b3d9.dl.dropboxusercontent.com/cd/0/inline/AvMnn8mXliVnniO6xr7k2eh8buC2ZRX3aDp37CU1Ti0KPF8Dw5Cr

In [0]:
# extract embeddings
embedding_cadastrado = []
embedding_novo = [] 
for amostra in  amostras_cadastradas:
  embedding_cadastrado.append([encoding_model.predict([amostra[0]])[0],amostra[1]]) # estrutura [embedding,id do locutor]
  
for amostra in  amostras_novas:
  embedding_novo.append([encoding_model.predict([amostra[0]])[0],amostra[1]]) 
  
  
  
  

In [10]:
import math
import numpy as np 
acertou = 0 
posI = 0

for j in range(len(embedding_novo)):
  #print(j,' de ',len(embedding_novo))
  menordist = math.inf
  i =0
                
  while i < len(embedding_cadastrado):
    distancia = np.sqrt(sum([(xi-yi)**2 for xi,yi in zip(embedding_novo[j][0],embedding_cadastrado[i][0])]))#distancia euclediana
    if distancia <  menordist:
      menordist = distancia
      posI= i
    i=i+1  
            
            
            
  if(embedding_cadastrado[posI][1] == embedding_novo[j][1]):
    acertou = acertou +1

print('numero de acertos:', acertou)               

           
          


numero de acertos: 314


In [11]:
acc  = acertou/len(embedding_novo)
print("Acuracia: ",acc)
print('De ',len(embedding_novo), " classificou corretamente ", acertou, 'amostras considerando ', len(embedding_cadastrado),'locutores cadastrados')

Acuracia:  0.7566265060240964
De  415  classificou corretamente  314 amostras considerando  20 locutores cadastrados
